In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
#Wczytuję dataset do pandas DataFrame
hb = pd.read_csv('hotel_bookings.csv.zip')

In [3]:
hb.shape

(119390, 32)

In [4]:
hb.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [5]:
hb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [6]:
#Sprawdzam w których kolumnach są brakujące dane
hb.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [7]:
#Sprawdzam udział brakujących danych w całym całym zbiorze dla danej kolumny
hb.isnull().sum()[hb.isnull().sum()!=0]/hb.shape[0]

children    0.000034
country     0.004087
agent       0.136862
company     0.943069
dtype: float64

In [8]:
#Kolumna 'company' ma ponad 94% brakujących rekordów. Decyduje się ją usunąć.
#Usuwam też kolumny 'reservation_status_date', 'arrival_date_year' ponieważ moim zdaniem nie powinny mieć wływu na predykcje.
#Usuwam również kolumnę 'reservation_status' - jest w niej informacja o anulowaniu rezerwacji, więc pokrywa się z naszą zmienną objaśnianą
hb.drop(columns = ['reservation_status_date', 'reservation_status', 'company', 'arrival_date_year'], inplace=True)

In [9]:
hb.children.value_counts()

0.0     110796
1.0       4861
2.0       3652
3.0         76
10.0         1
Name: children, dtype: int64

In [10]:
#Dominującą liczbą posiadanych dzieci jest 0, dlatego wstawiam 0 w miejsce n/a
hb.children.fillna(value=0, inplace=True)

In [11]:
hb.country.value_counts()

PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
PLW        1
LCA        1
SMR        1
UMI        1
NAM        1
Name: country, Length: 177, dtype: int64

In [12]:
#Krajem który najczęściej występuje jest Portugalia, dlatego braku uzupełnię 'PRT'
hb.country.fillna(value='PRT', inplace=True)

In [13]:
hb.agent.value_counts()

9.0      31961
240.0    13922
1.0       7191
14.0      3640
7.0       3539
         ...  
213.0        1
433.0        1
197.0        1
367.0        1
337.0        1
Name: agent, Length: 333, dtype: int64

In [14]:
#Dla kolumny 'agent' uzupełnię braki mediana
hb_median = hb.agent.median()
hb.agent.fillna(value=hb_median, inplace=True)

In [15]:
#Sprawdzam czy wszystkie braki w danych zostały wypełnione
hb.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests   

In [16]:
#Upraszczam kolumnę children do wartości 0 lub 1
def zamiana_children(i):
    if i > 0:
        return 1
    else: return 0
    
children_0_1 = hb.children.apply(zamiana_children)
hb.children = children_0_1

In [17]:
#Upraszczam kolumnę babies do wartości 0 lub 1
def zamiana_babies(i):
    if i > 0:
        return 1
    else: return 0
    
babies_0_1 = hb.babies.apply(zamiana_babies)
hb.babies = babies_0_1

In [18]:
#Dla kolumny 'meal' zamieniam kategorie na wartości. Zgodnie z opisem datasetu 'SC'='Undefined'.
meal_dict = {'SC' : 0,
            'Undefined' : 0,
            'BB' : 1,
            'HB' : 2,
            'FB' : 3}
meal_rank = hb.meal.map(meal_dict)
hb.meal = meal_rank

In [19]:
#Dodaję nową kolumnę, która sprawdza czy zarezerwowany przez użytkownika typ pokoju został zmieniony
bool_to_int = {False : 0,
              True : 1}

hb['is_type_room_changed'] = (hb.reserved_room_type != hb.assigned_room_type).map(bool_to_int)

In [20]:
#Zamieniam wartosci kategoryczne w kolumnie deposit ranking 0, 1, 2.
deposit_dict = {'No Deposit' : 0,
            'Refundable' : 1,
            'Non Refund' : 2}

deposit_rank = hb.deposit_type.map(deposit_dict)
hb.deposit_type = deposit_rank

In [21]:
hb.dtypes[hb.dtypes=='object']

hotel                   object
arrival_date_month      object
country                 object
market_segment          object
distribution_channel    object
reserved_room_type      object
assigned_room_type      object
customer_type           object
dtype: object

In [22]:
#Zamieniam pozostałe zmienne katogoryczne z wykorzystaniem get_dummies
hb_gd = pd.get_dummies(hb).copy()

In [23]:
hb_gd.shape

(119390, 251)

# Podział na X, y oraz na zbiór treningowy i testowy

In [24]:
#Oddzielam zmienną objaśnianą od zmiennych objaśniających
y = hb_gd['is_canceled']
X = hb_gd.drop(columns=['is_canceled'], axis=1)

In [25]:
#Dzielę mój data set na zbór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## DecisionTreeClassifier

In [26]:
#Uruchamian instancję DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_depth=43, criterion='entropy', random_state=42)

In [27]:
#Dopasowuję model do danych. Etap uczenia
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=43, random_state=42)

In [28]:
#Dokonuję predykcji na danych testowych
y_pred = dt_clf.predict(X_test)

In [29]:
#Sprawdzam jakość modelu
accuracy_score(y_test, y_pred)

0.8656922690342574

In [30]:
confusion_matrix(y_test, y_pred)

array([[13263,  1644],
       [ 1563,  7408]], dtype=int64)

## RandomForestClassifier

In [35]:
#Uruchamian instancję RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=42)

In [36]:
#Dopasowuję model do danych. Etap uczenia
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [37]:
#Dokonuję predykcji na danych testowych
y_pred = rf_clf.predict(X_test)

In [38]:
#Sprawdzam jakość modelu
accuracy_score(y_test, y_pred)

0.8967250188457995

In [39]:
confusion_matrix(y_test,y_pred)

array([[14066,   841],
       [ 1625,  7346]], dtype=int64)